In [8]:
import os
import pickle
import pandas as pd
import requests
from bs4 import BeautifulSoup
os.listdir('To_Predict/2_liga/')[1][-3:]

'csv'

Create a data processing pipeline

In [9]:
def pipeline(pickle_dict):
    dict = {"link":[], "Elo_home":[], "Elo_away":[]}
    for key in pickle_dict.keys():
        dict["link"].append(key)
        dict["Elo_home"].append(pickle_dict[key]["Elo_home"])
        dict['Elo_away'].append(pickle_dict[key]['Elo_away'])
    df = pd.DataFrame(data = dict)
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    n_rows = df.shape[0]
    for i in df.index:
        url = df.loc[i,'link']
        req = requests.get(url+'/table')
        soup = BeautifulSoup(req.content, 'html.parser')
        home = soup.select('p[itemprop=homeTeam]')[0].text[1:-1]
        away = soup.select('p[itemprop=awayTeam]')[0].text[1:-1]
        home_score = soup.select('span[class=r1]')[0].get_text()
        away_score = soup.select('span[class=r2]')[0].get_text()
        home_GF = 0
        home_GA = 0
        away_GF = 0
        away_GA = 0
        for html in soup.select('.row-body.marked'):
            if html.select('span[class=team-name]')[0].get_text() == home:
                home_GF = html.select('td[data-content-tab=tab_cc_total]')[5].get_text()
                home_GA = html.select('td[data-content-tab=tab_cc_total]')[6].get_text()
            elif html.select('span[class=team-name]')[0].get_text() == away:
                away_GF = html.select('td[data-content-tab=tab_cc_total]')[5].get_text()
                away_GA = html.select('td[data-content-tab=tab_cc_total]')[6].get_text()
        df.loc[i, 'home_GF'] = home_GF
        df.loc[i, 'home_GA'] = home_GA
        df.loc[i, 'away_GF'] = away_GF
        df.loc[i, 'away_GA'] = away_GA
        if home_score > away_score:
            df.loc[i, 'result'] = 1
        elif home_score == away_score:
            df.loc[i, 'result'] = 0
        else:
            df.loc[i, 'result'] = -1
    return df

Merge all data from pickle files

retrieve all pickle file directories

In [10]:
def search_pkl_files(directory):
    l = []
    for dir in os.listdir(directory):
        if os.path.isdir(directory+'/'+dir):
            l = l+search_pkl_files(directory+'/'+dir)
        elif os.path.isfile(directory+'/'+dir):
            if dir[-3:] == 'pkl':
                l.append(directory+'/'+dir)
    return l

In [11]:
list = search_pkl_files('To_Predict')
list

['To_Predict/championship/elo_dict.pkl',
 'To_Predict/eerste_divisie/elo_dict.pkl',
 'To_Predict/primeira_liga/elo_dict.pkl',
 'To_Predict/ligue_1/elo_dict.pkl',
 'To_Predict/segunda_division/elo_dict.pkl',
 'To_Predict/2_liga/elo_dict.pkl',
 'To_Predict/serie_a/elo_dict.pkl',
 'To_Predict/bundesliga/elo_dict.pkl',
 'To_Predict/primera_division/elo_dict.pkl',
 'To_Predict/ligue_2/elo_dict.pkl',
 'To_Predict/premier_league/elo_dict.pkl',
 'To_Predict/eredivisie/elo_dict.pkl',
 'To_Predict/segunda_liga/elo_dict.pkl',
 'To_Predict/serie_b/elo_dict.pkl']

In [12]:
dict = {}
for dir in list:
    elo_dict = pickle.load(open(dir, 'rb'))
    dict = {**dict, **elo_dict}
dict

{'https://www.besoccer.com/match/hull-city/huddersfield-town-fc/202213336': {'Elo_home': '70',
  'Elo_away': '55'},
 'https://www.besoccer.com/match/blackpool-fc/nottingham-forest-fc/202213332': {'Elo_home': '39',
  'Elo_away': '57'},
 'https://www.besoccer.com/match/afc-bournemouth/bristol-city-fc/202213329': {'Elo_home': '67',
  'Elo_away': '53'},
 'https://www.besoccer.com/match/barnsley-fc/reading-fc/202213330': {'Elo_home': '50',
  'Elo_away': '60'},
 'https://www.besoccer.com/match/cardiff-city-fc/swansea-city-afc/202213333': {'Elo_home': '57',
  'Elo_away': '75'},
 'https://www.besoccer.com/match/coventry-city/blackburn-rovers-fc/202213334': {'Elo_home': '43',
  'Elo_away': '60'},
 'https://www.besoccer.com/match/derby-county-fc/preston-north-end/202213335': {'Elo_home': '61',
  'Elo_away': '58'},
 'https://www.besoccer.com/match/luton-town-fc/millwall-fc/202213337': {'Elo_home': '36',
  'Elo_away': '48'},
 'https://www.besoccer.com/match/peterborough-u/middlesbrough-fc/20221333

In [13]:
df = pipeline(dict)
df.head()
df.drop(columns=['index', 'link'])
df.to_csv('to_predict.csv')